In [12]:
import numpy as np
from GroupingAlgorithms import *
# Importing standard Qiskit libraries
from qiskit import IBMQ
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.drivers import PyQuanteDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit.opflow import converters
from qiskit.opflow.primitive_ops import Z2Symmetries

In [52]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 
backend = provider.get_backend('ibmq_paris')
WC_paris = get_backend_conectivity(backend)

ibmqfactory.load_account:WARNING:2021-05-28 21:32:20,942: Credentials are already in use. The existing account in the session will be replaced.


In [73]:
molecules = [ 
#             'H .0 .0 .0; H .0 .0 0.761'  #0.88
#              ,
             'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'
#              ,
#             'H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339'
#              ,
#             'O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0'
# #              ,
#              'H 0.0 0.0 0.0; F 0.0 0.0 0.995'
             ,
#             'Cl 0.0 0.0 0.0; H 0.0 0.0 1.0' 
            ]


for molecule in molecules :
    
    print( 'molecule: '+molecule )

    driver = PyQuanteDriver(molecule)
    qmolecule = driver.run()
    problem = ElectronicStructureProblem(driver)

    # Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

    # Hamiltonian
    main_op = second_q_ops[0]

    # Setup the mapper and qubit converter
    mapper_type = 'JordanWignerMapper'

    if mapper_type == 'ParityMapper':
        mapper = ParityMapper()
    elif mapper_type == 'JordanWignerMapper':
        mapper = JordanWignerMapper()
    elif mapper_type == 'BravyiKitaevMapper':
        mapper = BravyiKitaevMapper()

    converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

    # The fermionic operators are mapped to qubit operators
    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    qubit_op   = converter.convert(main_op, num_particles=num_particles)

    num_qubits = qubit_op.num_qubits
    WC=list(range(num_qubits))
    WC=list(permutations(list(range(num_qubits)),2))

    print('number of qubits:', num_qubits)

    paulis, coeff, label = Label2Chain(qubit_op)
    print( 'number of Paulis:', len(coeff) )
    
    Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
    print( 'number of TPB groups:', len(Groups_tpb) )
        
    Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
    print('number of Bell groups:', len(Groups_bell) )
    
    Groups_2Q0, Measurements_2Q0 = grouping( paulis,[3,2,1,4,9,8,7,6,5],WC)
    print('number of 2Q groups TPB:', len(Groups_2Q0) )
    
    Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC)
    print('number of 2Q groups Bell-χ:', len(Groups_2Q1) )
    
    Groups_2Q2, Measurements_2Q2 = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC)
    print('number of 2Q groups Ω:', len(Groups_2Q2) )
    
    Groups_bell_paris, Measurements_bell_paris = grouping( paulis,[4,3,1,2],WC_paris)
    print('number of Bell groups for ibm_paris:', len(Groups_bell) )
    
    Groups_2Q0_paris, Measurements_2Q0 = grouping( paulis,[3,2,1,4,9,8,7,6,5],WC_paris)
    print('number of 2Q groups TPB for ibm_paris:', len(Groups_2Q0) )
    
    Groups_2Q1_paris, Measurements_2Q1_paris = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC_paris)
    print('number of 2Q groups Bell-χ for ibm_paris:', len(Groups_2Q1) )
    
    Groups_2Q2_paris, Measurements_2Q2_paris = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC_paris)
    print('number of 2Q groups Ω for ibm_paris:', len(Groups_2Q2) )    
    
    print('----------------------------------------')


molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619
number of qubits: 12
number of Paulis: 631
number of TPB groups: 136


D:\Saco_CSIC\Programas\Github\Hardware-efficient-variational-quantum-eigensolver-with-entangled-measurements\Codes\GroupingAlgorithms.py:118: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if (list(Vi[[per]]) in Comp[Eps]) and (list(Vj[[per]]) in Comp[Eps]):


number of Bell groups: 39
number of 2Q groups TPB: 54
number of 2Q groups Bell-χ: 39
number of 2Q groups Ω: 64
number of Bell groups for ibm_paris: 39
number of 2Q groups TPB for ibm_paris: 54
number of 2Q groups Bell-χ for ibm_paris: 39
number of 2Q groups Ω for ibm_paris: 64
----------------------------------------


In [101]:
Measurements_bell[2]

[[4, [1, 5]],
 [4, [7, 10]],
 [4, [3, 4]],
 [4, [6, 11]],
 [3, [2]],
 [3, [0]],
 [3, [8]],
 [3, [9]]]

In [102]:
Measurements_bell_paris[2]

[[4, [8, 9]],
 [3, [10]],
 [2, [6]],
 [2, [11]],
 [4, [2, 3]],
 [3, [4]],
 [1, [0]],
 [1, [5]],
 [1, [1]],
 [2, [7]]]

In [98]:
Test = []
for groups in Measurements_bell:
    for idx in groups:
        if idx[0] == 4 :
            if tuple(idx[1]) in WC_paris:
                Test.append(1)
            else:
                Test.append(0)
                
np.prod(Test)

0

In [96]:
Test = []
for groups in Measurements_bell_paris:
    for idx in groups:
        if idx[0] == 4 :
            if tuple(idx[1]) in WC_paris:
                Test.append(1)
            else:
                Test.append(0)
                
np.prod(Test)

1

In [23]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PyQuanteDriver(molecule)
qmolecule = driver.run()
freezeCoreTransfomer = FreezeCoreTransformer(freeze_core=True,remove_orbitals= [3,4])
problem = ElectronicStructureProblem(driver,q_molecule_transformers=[freezeCoreTransfomer])

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

# Setup the mapper and qubit converter
mapper_type = 'ParityMapper'
mapper = ParityMapper()

converter = QubitConverter(mapper=mapper,two_qubit_reduction=True, z2symmetry_reduction=[1,1],) #1] 

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)

num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

qubit_op = converter.convert(main_op, num_particles=num_particles)

num_qubits = qubit_op.num_qubits
num_qubits = qubit_op.num_qubits
WC=list(range(num_qubits))
WC=list(permutations(list(range(num_qubits)),2))

print( num_qubits )
print( qubit_op )

4
-0.20316612481217325 * IIII
+ (-0.365258527098598-1.3877787807814457e-17j) * ZIII
+ 0.09275974420404098 * IZII
- 0.2118902053549851 * ZZII
+ (0.365258527098598+2.7755575615628914e-17j) * IIZI
- 0.11384420116816225 * ZIZI
+ 0.11395335131674317 * IZZI
+ (-0.06043919280467715-3.469446951953614e-18j) * ZZZI
+ (-0.09275974420404105-3.469446951953614e-18j) * IIIZ
+ 0.11395335131674318 * ZIIZ
+ (-0.12274347556009738-6.938893903907228e-18j) * IZIZ
+ 0.056287757667526306 * ZZIZ
+ (-0.2118902053549851-6.938893903907228e-18j) * IIZZ
+ 0.06043919280467715 * ZIZZ
- 0.056287757667526306 * IZZZ
+ 0.0846016988633049 * ZZZZ
+ 0.019390023793252498 * XIII
+ (-0.019390023793252498+8.673617379884035e-19j) * XZII
- 0.010951534973539137 * XIZI
+ 0.010951534973539137 * XZZI
+ 0.01277809801936714 * XIIZ
- 0.01277809801936714 * XZIZ
- 0.00900252139905185 * XIZZ
+ 0.00900252139905185 * XZZZ
- 0.002936887226605865 * IXII
- 0.0029368872266058656 * ZXII
+ 0.010679892682752462 * IXZI
+ 0.010679892682752462 * ZXZI


In [42]:
paulis, coeff, label = Label2Chain(qubit_op)
print( 'number of Paulis:', len(coeff) )

Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
print( 'number of TPB groups:', len(Groups_tpb) )

Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
print('number of Bell groups:', len(Groups_bell) )

Groups_2Q0, Measurements_2Q0 = grouping( paulis,[3,2,1,4,9,8,7,6,5],WC)
print('number of 2Q groups TPB:', len(Groups_2Q0) )

Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC)
print('number of 2Q groups Bell-χ:', len(Groups_2Q1) )

Groups_2Q2, Measurements_2Q2 = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC)
print('number of 2Q groups Ω:', len(Groups_2Q2) )

number of Paulis: 100
number of TPB groups: 25
number of Bell groups: 9
number of 2Q groups TPB: 11
number of 2Q groups Bell-χ: 11
number of 2Q groups Ω: 11


In [43]:
Groups_bell

[[99, 39, 87, 97, 98, 15, 38, 83, 96, 33, 84, 3, 12, 32, 80, 0],
 [59, 37, 57, 47, 58, 11, 36, 43, 56, 44, 8, 40],
 [79, 35, 77, 67, 78, 7, 34, 63, 76, 64, 4, 60],
 [91, 86, 90, 23, 89, 14, 22, 82, 88, 17, 2, 16],
 [95, 85, 94, 31, 93, 13, 30, 81, 92, 25, 1, 24],
 [51, 21, 46, 49, 50, 10, 20, 42, 48],
 [55, 29, 45, 53, 54, 9, 28, 41, 52],
 [71, 19, 66, 69, 70, 6, 18, 62, 68],
 [75, 27, 65, 73, 74, 5, 26, 61, 72]]

In [ ]:
#0-1-2-3
#  |
#  4

In [45]:
Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
print( 'number of TPB groups:', len(Groups_tpb) )

Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
print('number of Bell groups:', len(Groups_bell) )

Groups_2Q0, Measurements_2Q0 = grouping( paulis,[3,2,1,4,9,8,7,6,5],WC)
print('number of 2Q groups TPB:', len(Groups_2Q0) )

Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC)
print('number of 2Q groups Bell-χ:', len(Groups_2Q1) )

Groups_2Q2, Measurements_2Q2 = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC)
print('number of 2Q groups Ω:', len(Groups_2Q2) )

number of TPB groups: 25
number of Bell groups: 9
number of 2Q groups TPB: 11
number of 2Q groups Bell-χ: 11
number of 2Q groups Ω: 11


In [46]:
Groups_bell

[[99, 39, 87, 97, 98, 15, 38, 83, 96, 33, 84, 3, 12, 32, 80, 0],
 [59, 37, 57, 47, 58, 11, 36, 43, 56, 44, 8, 40],
 [79, 35, 77, 67, 78, 7, 34, 63, 76, 64, 4, 60],
 [91, 86, 90, 23, 89, 14, 22, 82, 88, 17, 2, 16],
 [95, 85, 94, 31, 93, 13, 30, 81, 92, 25, 1, 24],
 [51, 21, 46, 49, 50, 10, 20, 42, 48],
 [55, 29, 45, 53, 54, 9, 28, 41, 52],
 [71, 19, 66, 69, 70, 6, 18, 62, 68],
 [75, 27, 65, 73, 74, 5, 26, 61, 72]]

In [50]:
Measurements_bell

[[[4, [2, 3]], [4, [0, 1]]],
 [[1, [2]], [3, [3]], [4, [0, 1]]],
 [[1, [3]], [3, [2]], [4, [0, 1]]],
 [[1, [0]], [3, [1]], [4, [2, 3]]],
 [[1, [1]], [3, [0]], [4, [2, 3]]],
 [[1, [2]], [1, [0]], [3, [3]], [3, [1]]],
 [[1, [2]], [1, [1]], [3, [3]], [3, [0]]],
 [[1, [3]], [1, [0]], [3, [2]], [3, [1]]],
 [[1, [3]], [1, [1]], [3, [2]], [3, [0]]]]

In [51]:
#0-1-2-3
#  |
#  4